# Alumno: Gerardo de Miguel González

1.- Crea una función que genere dos muestras que se distribuyen según dos gaussianas distintas. La función recibirá como valores de entrada: el número N de puntos a generar para cada categoría, y mu1, sigma1, mu2, sigma2 que son los correspondientes parámetros de las dos gaussianas. Como output devolverá un valor con longitud 2N que contenga la muestra x generada, y otro vector de longitud 2N que contenga 0 o 1 en función de la categoría asociada a ese elemento.

In [1]:
muestras <- function(n = 10,
                     mu_1 = 0,
                     sigma_1 = 1, 
                     mu_2 = 0,
                     sigma_2 = 1) {
    x_1 <- rnorm(n = n, mean = mu_1, sd = sigma_1)
    x_2 <- rnorm(n = n, mean = mu_2, sd = sigma_2)
    y_1 <- rep(0,length(x_1))
    #::GMG::http://www.cookbook-r.com/Basics/Making_a_vector_filled_with_values/
    y_2 <- rep(1, length(x_2))
    R <- t(cbind(rbind(x_1,y_1),rbind(x_2,y_2)))
    # https://stackoverflow.com/questions/32658793/how-to-remove-column-names-from-a-matrix-in-r
    colnames(R) <- NULL
    # http://www.cookbook-r.com/Manipulating_data/Randomizing_order/
    # http://r.789695.n4.nabble.com/Permuting-rows-of-a-matrix-td3298688.html
    return(R[sample(nrow(R)),])
}

In [2]:
# https://stats.stackexchange.com/questions/86285/random-number-set-seedn-in-r
set.seed(1)
Z <- muestras(n = 20)
dim(Z)
str(Z)
print(Z)

[1] 40  2

 num [1:40, 1:2] 0.9438 -1.4708 -0.0449 -0.6212 1.1 ...
 - attr(*, "dimnames")=List of 2
  ..$ : NULL
  ..$ : NULL
             [,1] [,2]
 [1,]  0.94383621    0
 [2,] -1.47075238    1
 [3,] -0.04493361    0
 [4,] -0.62124058    0
 [5,]  1.10002537    1
 [6,]  0.73832471    0
 [7,]  0.61982575    1
 [8,]  0.32950777    0
 [9,] -1.37705956    1
[10,]  0.38767161    1
[11,] -0.10278773    1
[12,]  0.18364332    0
[13,]  0.76317575    1
[14,] -1.98935170    1
[15,]  0.91897737    1
[16,]  0.59390132    0
[17,]  1.51178117    0
[18,] -0.30538839    0
[19,] -0.41499456    1
[20,] -0.39428995    1
[21,] -2.21469989    0
[22,]  0.48742905    0
[23,]  1.35867955    1
[24,] -0.01619026    0
[25,] -0.15579551    1
[26,]  1.59528080    0
[27,] -0.47815006    1
[28,]  0.82122120    0
[29,]  0.38984324    0
[30,] -0.05612874    1
[31,]  0.07456498    1
[32,] -0.05931340    1
[33,] -0.83562861    0
[34,]  1.12493092    0
[35,] -0.62645381    0
[36,] -0.82046838    0
[37,]  0.41794156    1
[38,]  0.78

2.- Crea una función que calcule el valor de la sigmoide para un valor de entrada Z. 

::GMG::https://helloacm.com/quick-r-tutorial-how-to-plot-sigmoid-function-using-r/

In [3]:
#::GMG::Función sigmoide
sigmoid = function(z) {
   1 / (1 + exp(-z))
}

In [4]:
#::GMG::Dibujo la función que he creado (opcional)
#z <- seq(-5, 5, 0.01)
#plot(z, sigmoid(z), col='red')

3.- Crea una función que calcule el valor de la función de Loss y que reciba como entrada “x” e “y” y los parámetros del modelo que vamos a asumir: z = a + b x (es decir, a y b). 

In [5]:
#::GMG::Función de coste de regresión logística con sigmoid
#       siguiendo la notación de loa apuntes, slide 10, Chapter 4
#       Ponemos los coeficientes en un vector de alphas
#       alpha_0, alpha_1, ..., alpha_p 
#       alpha_0 + alpha_1 * x_1 + ... + alpha_p * x_p
#       x es la matriz de features (1, x_i)
loss <- function(x, y, alpha) {
    s <- sigmoid(z = x %*% alpha)
    return(- sum(y * log(s) + (1-y) * log(1-s)) / nrow(x))
}

4.- Crea una función que devuelva el gradiente de la función de Loss y que reciba como entrada “x” e “y” y los parámetros (a, b) del modelo que vamos a asumir.

In [6]:
#::GMG::Gradiente del coste según los apuntes (slide 11, Chapter 4)
gradiente <- function(x, y, alpha) {
    return((t(t(sigmoid(z = x %*% alpha) - y) %*% x)) / nrow(x))
}

5.- Generar un par de vectores “x”, “y” con N = 100, mu1 = 2, mu2 = 6, sigma1 = 1 y sigma2 = 1.

In [7]:
# https://stats.stackexchange.com/questions/86285/random-number-set-seedn-in-r
set.seed(123)

In [8]:
Z <- muestras(n = 100, mu_1 = 2, mu_2 = 6, sigma_1 = 1, sigma_2 = 1)
dim(Z)
str(Z)

[1] 200   2

 num [1:200, 1:2] 4.749 2.153 5.542 5.944 0.779 ...
 - attr(*, "dimnames")=List of 2
  ..$ : NULL
  ..$ : NULL


6.- Calcular la función de coste y el gradiente para (a = 0, b = 0). Actualizar los valores de a y b de manera que (a, b)_nuevos = (a, b)_viejos + lambda * gradiente. Repite 3 o 4 cuatro veces y observa los valores de la función de coste. Intenta encontrar el mínimo aproximadadamente. Interpreta los resultados.

In [9]:
#::GMG::Probamos el cálculo para unos valores iniciales de a y b
l <- loss(x = cbind(1, Z[,1]), y = Z[,2], alpha = c(0,0))
g <- gradiente(x = cbind(1, Z[,1]), y = Z[,2], alpha = c(0,0))
paste('coste', l)
paste('gradiente del coste', '(', g[1], g[2], ')')

[1] "coste 0.693147180559945"

[1] "gradiente del coste ( 0 -0.950511823340071 )"

In [10]:
#::GMG::Iteramos unas cuantas veces para encontrar un mínimo
lambda <- 0.5
alpha = c(0,0)
l <- c()
a_min <- list()
for (i in 1:60000) {
    l[i] <- loss(x = cbind(1, Z[,1]), y = Z[,2], alpha = alpha)
    g <- gradiente(x = cbind(1, Z[,1]), y = Z[,2], alpha = alpha)
    a_min[[i]] <- alpha <- alpha - lambda*g
}

In [11]:
paste('Encontré un mínimo en  la iteración', which.min(l) ,
      'con coste',  l[which.min(l)], 'y parámetros: (a=', 
      a_min[[which.min(l)]][1], ', b=', a_min[[which.min(l)]][2], ')'
     )

[1] "Encontré un mínimo en 55314 con coste 0.0243549029707509 y parámetros: (a= -29.7017819571011 , b= 7.18951911964683 )"